# Selecting areas of interest for our reporting - incidents after 2010, pipelines installed any year

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

/Users/ilenapeng/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
df = pd.read_csv('../data/processed/pipeline_incidents_2010_present_all_CLEAN.csv')
df.head()

,datafile_as_of,ff,significant,serious,report_number,supplemental_number,report_received_date,report_type,operator_id,name,operator_street_address,operator_city_name,operator_state_abbreviation,operator_postal_code,local_datetime,time_zone,daylight_savings_ind,iyear,location_street_address,location_city_name,location_county_name,location_state_abbreviation,location_postal_code,location_latitude,location_longitude,total_cost,total_cost_current,injury_ind,injure,num_pub_evacuated,fatal,cause,cause_details,narrative,unintentional_release,installation_year
0,2/28/22,NO,YES,NO,20100001,15047,3/11/10,SUPPLEMENTAL FINAL,15007,PACIFIC GAS & ELECTRIC CO,"PG&E - GAS OPERATIONS, REGULATORY COMPLIANCE 6...",SAN RAMON,CA,94583,2/13/10 23:35,NaN,NaN,2010,1617 EAST 9TH STREET,STOCKTON,SAN JOAQUIN,CA,95201,37.93188,-121.26133,102500,124764.082311,NO,0,0.0,0,OTHER OUTSIDE FORCE DAMAGE,OTHER OUTSIDE FORCE DAMAGE,A FIRE AT AN UNOCCUPIED HOME OCCURRED AT APPRO...,10.0,NaN
1,2/28/22,NO,NO,NO,20100002,15553,3/12/10,SUPPLEMENTAL FINAL,13730,NORTHERN INDIANA PUBLIC SERVICE CO,801 E 86TH AVENUE,MERRILLVILLE,IN,46410,2/16/10 10:38,NaN,NaN,2010,3835 SANDPIPER COVE RUN,SOUTH BEND,ST. JOSEPH,IN,46628,41.72931,-86.27531,57500,68334.121206,NO,0,20.0,0,EXCAVATION DAMAGE,EXCAVATION DAMAGE BY THIRD PARTY,"CONTRACTOR BORING ELECTRIC LINE PARALLEL TO 3""...",310.0,1990-01-01
2,2/28/22,NO,NO,NO,20100003,15023,3/15/10,SUPPLEMENTAL FINAL,31840,UNISOURCE ENERGY SERVICES,2901 W. SHAMRELL BLVD.,FLAGSTAFF,AZ,86001,2/22/10 20:57,NaN,NaN,2010,8725 COUNTRY CLUB DRIVE,PINETOP,NAVAJO,AZ,85935,34.10934,-109.86010,2648,3173.508982,NO,0,NaN,0,NATURAL FORCE DAMAGE,OTHER NATURAL FORCE DAMAGE,ON 2/22/10 PINETOP FIRE DEPARTMENT RESPONDED T...,14.0,1999-01-01
3,2/28/22,NO,YES,YES,20100004,15003,3/16/10,ORIGINAL FINAL,13710,NORTHERN ILLINOIS GAS CO,"P.O. BOX 190, RT 59 & I-88",AURORA,IL,60507,2/18/10 19:05,NaN,NaN,2010,359 HAWTHORNE CIRCLE,MOUNT PROSPECT,COOK,IL,60056,42.03668,-87.94307,22005,26781.407382,YES,1,NaN,0,CORROSION FAILURE,EXTERNAL CORROSION,CREW WAS DIGGING A HOLE TO FIX A SERVICE LINE ...,1.0,1976-01-01
4,2/28/22,NO,YES,YES,20100005,15639,3/22/10,SUPPLEMENTAL FINAL,6141,NEW MEXICO GAS COMPANY,"7120 WYOMING BLVD. NE STE, 20",ALBUQUERQUE,NM,87109,2/21/10 1:38,NaN,NaN,2010,303 COUNTY RD. 6100,KIRTLAND,SAN JUAN,NM,87417,36.74350,-108.33380,4843,5871.773935,YES,1,0.0,0,OTHER OUTSIDE FORCE DAMAGE,"DAMAGE BY CAR, TRUCK, OR OTHER MOTORIZED VEHIC...",A SAN JUAN COUNTY DEPUTY SHERIFF WAS IN PURSUI...,5625.0,NaN


## Cities with most pipeline accidents

In [3]:
df.location_city_name.value_counts().head(20)

NEW YORK                     30
Not Within a Municipality    12
BROOKLYN                     12
SAN JOSE                     12
LAS VEGAS                    12
DETROIT                      12
HOUSTON                      10
DALLAS                        9
LOS ANGELES                   9
OMAHA                         8
KANSAS CITY                   8
COLUMBUS                      7
BALTIMORE                     7
SAN DIEGO                     7
CLEVELAND                     7
ATLANTA                       7
BRONX                         7
CHICAGO                       7
CORPUS CHRISTI                6
PHILADELPHIA                  6
Name: location_city_name, dtype: int64

## Cities with highest total costs for pipeline accidents (adjusted for inflation)
total_cost variable would give us the estimated total cost

In [4]:
df_highest_cost = df.groupby(['location_city_name', 'location_state_abbreviation']).total_cost_current.agg(['sum', 'count']).reset_index()
df_highest_cost.sort_values('sum', ascending=False).head(20)

,location_city_name,location_state_abbreviation,sum,count
457,LAWRENCE,MA,1.687948e+09,4
637,PARADISE,CA,3.302340e+08,1
535,MINNEAPOLIS,MN,5.221857e+07,3
241,DURHAM,NC,3.693140e+07,4
415,KANSAS CITY,MO,2.407942e+07,8
812,SUN PRAIRIE,WI,2.177038e+07,1
652,PHOENIX,AZ,2.015115e+07,3
480,LONG GROVE,IL,1.763341e+07,1
21,AMERICAN CANYON,CA,1.737451e+07,1
422,KENWOOD,CA,1.703302e+07,1


## Cities with highest fatalities and injuries

In [5]:
df_injure_fatal = df.groupby(['location_city_name', 'location_state_abbreviation']).agg({'fatal' : ['sum'], 'injure' : ['sum', 'count']})
df_injure_fatal.head(2)

fatal injure      
                                                 sum    sum count
location_city_name location_state_abbreviation                   
ABERDEEN           ID                              0      0     1
                   SD                              0      0     1

In [6]:
# remove multi index
df_injure_fatal.columns = df_injure_fatal.columns.droplevel()
df_injure_fatal = df_injure_fatal.reset_index()

# rename columns
df_injure_fatal.columns = ['location_city_name',
                           'location_state_abbreviation',
                           'fatal_sum',
                           'injure_sum',
                           'accident_count']


df_injure_fatal.head()

,location_city_name,location_state_abbreviation,fatal_sum,injure_sum,accident_count
0,ABERDEEN,ID,0,0,1
1,ABERDEEN,SD,0,0,1
2,ACAMPO,CA,0,0,1
3,ADAIRSVILLE,GA,0,1,1
4,ADAMS,NY,0,0,1


In [7]:
# Cities with most injuries
df_injure_fatal.sort_values('injure_sum', ascending=False).head(20)

,location_city_name,location_state_abbreviation,fatal_sum,injure_sum,accident_count
573,NEW YORK,NY,9,52,30
772,SILVER SPRING,MD,7,33,2
457,LAWRENCE,MA,1,27,4
795,SPRINGFIELD,MA,0,17,1
232,DETROIT,MI,1,16,12
270,EWING,NJ,1,10,1
597,NYACK,NY,0,8,2
160,CHICAGO,IL,1,7,7
651,PHILADELPHIA,PA,3,7,6
463,LEBANON,OH,0,7,1


In [8]:
# Cities with most fatalities
df_injure_fatal.sort_values('fatal_sum', ascending=False).head(20)

,location_city_name,location_state_abbreviation,fatal_sum,injure_sum,accident_count
573,NEW YORK,NY,9,52,30
772,SILVER SPRING,MD,7,33,2
16,ALLENTOWN,PA,5,3,1
807,STUDIO CITY,CA,3,0,2
651,PHILADELPHIA,PA,3,7,6
618,OMAHA,NE,3,2,8
388,HYATTSVILLE,MD,2,4,3
50,AUSTIN,TX,2,3,4
888,WAYNE,MI,2,2,1
535,MINNEAPOLIS,MN,2,3,3


## Incidents with highest fatalities and injuries

In [9]:
# Specific incidents with most fatalities
df.sort_values('fatal', ascending=False).head(10)

,datafile_as_of,ff,significant,serious,report_number,supplemental_number,report_received_date,report_type,operator_id,name,operator_street_address,operator_city_name,operator_state_abbreviation,operator_postal_code,local_datetime,time_zone,daylight_savings_ind,iyear,location_street_address,location_city_name,location_county_name,location_state_abbreviation,location_postal_code,location_latitude,location_longitude,total_cost,total_cost_current,injury_ind,injure,num_pub_evacuated,fatal,cause,cause_details,narrative,unintentional_release,installation_year
450,2/28/22,NO,YES,YES,20140031,16185,4/11/14,SUPPLEMENTAL FINAL,2704,CONSOLIDATED EDISON CO OF NEW YORK,4 IRVING PLACE,NEW YORK,NY,10003,3/12/14 9:31,NaN,NaN,2014,1644/1646 PARK AVENUE,NEW YORK,NEW YORK,NY,10035-4645,40.799726,-73.943401,2960200,3.339637e+06,YES,48,100.0,8,NATURAL FORCE DAMAGE,"EARTH MOVEMENT, NOT DUE TO HEAVY RAINS/FLOODS","DESCRIPTION OF INCIDENT: ON MARCH 12, 2014 ...",30.000,2011-01-01
1375,2/28/22,NaN,YES,YES,20100070,17044,10/13/10,SUPPLEMENTAL FINAL,15007,PACIFIC GAS & ELECTRIC CO,"PG&E - GAS OPERATIONS, REGULATORY COMPLIANCE 6...",SAN RAMON,CA,94583,9/9/10 18:11,NaN,NaN,2010,NaN,NaN,NaN,NaN,NaN,37.622457,-122.441898,558363000,6.794712e+08,YES,51,108.0,8,MATERIAL FAILURE OF PIPE OR WELD,ORIGINAL MANUFACTURING-RELATED (NOT GIRTH WELD...,"AT APPROXIMATELY 1811 HOURS ON SEPTEMBER 9, 20...",47600.000,1956-01-01
699,2/28/22,NO,YES,YES,20160086,31255,9/9/16,SUPPLEMENTAL,22182,WASHINGTON GAS LIGHT CO,"1000 MAINE AVENUE, SW",WASHINGTON,DC,20024,8/11/16 0:10,NaN,NaN,2016,8701 ARLISS ST,SILVER SPRING,MONTGOMERY COUNTY,MD,20901-3981,38.998431,-77.001087,1702454,1.880320e+06,YES,33,200.0,7,OTHER INCIDENT CAUSE,UNKNOWN,INVESTIGATION IS ONGOING UNDER THE DIRECTION O...,38.000,NaN
7020,2/28/22,NaN,YES,YES,20200256,36284,9/18/20,SUPPLEMENTAL FINAL,31618,ENTERPRISE PRODUCTS OPERATING LLC,1100 LOUISIANA STREET,HOUSTON,TX,77002,8/21/20 8:02,NaN,NaN,2020,NaN,NaN,NaN,NaN,NaN,27.816161,-97.468802,2023000,2.063344e+06,YES,4,0.0,5,EXCAVATION DAMAGE,EXCAVATION DAMAGE BY THIRD PARTY,"IN JUNE 2020, ENTERPRISE BECAME AWARE THAT ORI...",NaN,1968-01-01
134,2/28/22,NO,YES,YES,20110046,16729,3/10/11,SUPPLEMENTAL FINAL,20010,"UGI UTILITIES, INC",1 UGI DRIVE,DENVER,PA,17517,2/9/11 22:45,NaN,NaN,2011,N. 13TH & W. ALLEN STREETS,ALLENTOWN,LEHIGH,PA,18102-2104,40.607090,-75.487940,1666147,1.988285e+06,YES,3,500.0,5,NATURAL FORCE DAMAGE,TEMPERATURE,"ON FEBRUARY 9TH, AN EXPLOSION OCCURRED AT N. 1...",25.000,1928-01-01
2036,2/28/22,NaN,YES,YES,20150133,17104,11/6/15,SUPPLEMENTAL FINAL,19570,TRANSCONTINENTAL GAS PIPE LINE COMPANY,ONE WILLIAMS CENTER MD 43-4,TULSA,OK,74172,10/8/15 11:00,NaN,NaN,2015,NaN,NaN,NaN,NaN,NaN,29.603103,-90.874794,7545044,8.410351e+06,YES,1,0.0,4,INCORRECT OPERATION,DAMAGE BY OPERATOR OR OPERATOR'S CONTRACTOR NO...,WILLIAMS PERSONNEL AND CONTRACTORS WERE PERFOR...,0.000,1989-01-01
284,2/28/22,NO,YES,YES,20120082,15534,10/3/12,SUPPLEMENTAL FINAL,18484,SOUTHERN CALIFORNIA GAS CO,555 WEST FIFTH STREET,LOS ANGELES,CA,90013,9/9/12 3:45,NaN,NaN,2012,11412 VENTURA BLVD.,STUDIO CITY,LOS ANGELES,CA,91604,34.139668,-118.379288,13525,1.578472e+04,NO,0,0.0,3,OTHER OUTSIDE FORCE DAMAGE,"DAMAGE BY CAR, TRUCK, OR OTHER MOTORIZED VEHIC...",VEHICLE CARRYING THREE ADULTS TRAVELING AT HIG...,22.000,1986-01-01
1200,2/28/22,NO,YES,YES,20210006,36187,1/8/21,SUPPLEMENTAL FINAL,12390,METROPOLITAN UTILITIES DISTRICT,7350 WORLD COMMUNICATIONS DR,OMAHA,NE,68122,12/8/20 8:14,CENTRAL,YES,2020,4810 S 51ST STREET,OMAHA,DOUGLAS,NE,68117,41.210950,-95.994870,515505,5.257854e+05,YES,1,0.0,3,OTHER OUTSIDE FORCE DAMAGE,INTENTIONAL DAMAGE,"ON DECEMBER 8, 2020 AT APPROXIMATELY 0825HRS T...",1.795,1959-01-01
388,2/28/22,NO,YES,YES,20130096,15908,8/28/13,SUPPLEMENTAL FINAL,2748,CONSUMERS ENERGY CO,ONE ENERGY PLAZA,JACKSON,MI,49201,7/30/13 13:45,NaN,NaN,2013,3116 E. VIENNA ROAD,CLIO,GENESEE,MI,48420-9133,43.177701,-83.654776,361590,4.157734e+05,YES,1,0.0,2,EXCAVATION DAMAGE,EXCAVATION DAMAGE BY THIRD PARTY,AN E

In [10]:
# Specific incidents with most injuries
df.sort_values('injure', ascending=False).head(10)

,datafile_as_of,ff,significant,serious,report_number,supplemental_number,report_received_date,report_type,operator_id,name,operator_street_address,operator_city_name,operator_state_abbreviation,operator_postal_code,local_datetime,time_zone,daylight_savings_ind,iyear,location_street_address,location_city_name,location_county_name,location_state_abbreviation,location_postal_code,location_latitude,location_longitude,total_cost,total_cost_current,injury_ind,injure,num_pub_evacuated,fatal,cause,cause_details,narrative,unintentional_release,installation_year
1375,2/28/22,NaN,YES,YES,20100070,17044,10/13/10,SUPPLEMENTAL FINAL,15007,PACIFIC GAS & ELECTRIC CO,"PG&E - GAS OPERATIONS, REGULATORY COMPLIANCE 6...",SAN RAMON,CA,94583,9/9/10 18:11,NaN,NaN,2010,NaN,NaN,NaN,NaN,NaN,37.622457,-122.441898,558363000,6.794712e+08,YES,51,108.0,8,MATERIAL FAILURE OF PIPE OR WELD,ORIGINAL MANUFACTURING-RELATED (NOT GIRTH WELD...,"AT APPROXIMATELY 1811 HOURS ON SEPTEMBER 9, 20...",47600.0,1956-01-01
450,2/28/22,NO,YES,YES,20140031,16185,4/11/14,SUPPLEMENTAL FINAL,2704,CONSOLIDATED EDISON CO OF NEW YORK,4 IRVING PLACE,NEW YORK,NY,10003,3/12/14 9:31,NaN,NaN,2014,1644/1646 PARK AVENUE,NEW YORK,NEW YORK,NY,10035-4645,40.799726,-73.943401,2960200,3.339637e+06,YES,48,100.0,8,NATURAL FORCE DAMAGE,"EARTH MOVEMENT, NOT DUE TO HEAVY RAINS/FLOODS","DESCRIPTION OF INCIDENT: ON MARCH 12, 2014 ...",30.0,2011-01-01
699,2/28/22,NO,YES,YES,20160086,31255,9/9/16,SUPPLEMENTAL,22182,WASHINGTON GAS LIGHT CO,"1000 MAINE AVENUE, SW",WASHINGTON,DC,20024,8/11/16 0:10,NaN,NaN,2016,8701 ARLISS ST,SILVER SPRING,MONTGOMERY COUNTY,MD,20901-3981,38.998431,-77.001087,1702454,1.880320e+06,YES,33,200.0,7,OTHER INCIDENT CAUSE,UNKNOWN,INVESTIGATION IS ONGOING UNDER THE DIRECTION O...,38.0,NaN
921,2/28/22,NO,YES,YES,20180092,35384,10/13/18,SUPPLEMENTAL FINAL,1209,COLUMBIA GAS OF MASSACHUSETTS,4 TECHNOLOGY DRIVE,WESTBOROUGH,MA,01581,9/13/18 16:18,EASTERN,NO,2018,85 ANDOVER ST,LAWRENCE,ESSEX,MA,01843,42.697006,-71.175320,1586502068,1.684597e+09,YES,25,0.0,1,INCORRECT OPERATION,PIPELINE OR EQUIPMENT OVERPRESSURED,"ON SEPTEMBER 13, 2018, COLUMBIA GAS OF MASSACH...",678.0,1961-01-01
296,2/28/22,NO,YES,YES,20120098,34473,12/2/12,SUPPLEMENTAL FINAL,1209,COLUMBIA GAS OF MASSACHUSETTS,4 TECHNOLOGY DRIVE,WESTBOROUGH,MA,01581,11/23/12 17:25,NaN,NaN,2012,453 WORTHINGTON STREET,SPRINGFIELD,HAMPDEN,MA,01105,42.106680,-72.589386,1310300,1.534490e+06,YES,17,20.0,0,INCORRECT OPERATION,DAMAGE BY OPERATOR OR OPERATOR'S CONTRACTOR NO...,OPERATOR EMPLOYEE RESPONDED TO ODOR CALL IN BA...,26.0,1993-01-01
1972,2/28/22,NaN,YES,YES,20150068,17537,5/15/15,SUPPLEMENTAL FINAL,15007,PACIFIC GAS & ELECTRIC CO,"PG&E - GAS OPERATIONS, REGULATORY COMPLIANCE 6...",SAN RAMON,CA,94583,4/17/15 14:29,NaN,NaN,2015,NaN,NaN,NaN,NaN,NaN,36.842071,-119.929395,5978250,6.654366e+06,YES,13,0.0,1,EXCAVATION DAMAGE,EXCAVATION DAMAGE BY THIRD PARTY,"ON APRIL 17, 2015 AT 1433 HOURS, PG&E WAS NOTI...",7200.0,1962-01-01
448,2/28/22,NO,YES,YES,20140029,15842,4/3/14,ORIGINAL,15952,PUBLIC SERVICE ELECTRIC & GAS CO,80 PARK PLAZA M/C - T18,NEWARK,NJ,07102-4194,3/4/14 12:52,NaN,NaN,2014,28 CROCKETT LANE,EWING,MERCER,NJ,08628,40.275270,-74.832990,15087500,1.702063e+07,YES,10,NaN,1,EXCAVATION DAMAGE,EXCAVATION DAMAGE BY THIRD PARTY,"ON TUESDAY, MARCH 4, 2014, AT 11:39 AM, THE CO...",11.0,1989-01-01
375,2/28/22,NO,YES,YES,20130082,16720,7/3/13,SUPPLEMENTAL FINAL,14330,ORANGE & ROCKLAND UTILITY INC,ONE BLUE HILL PLAZA,PEARL RIVER,NY,10965,6/4/13 11:55,NaN,NaN,2013,177 SOUTH HIGHLAND AVENUE,NYACK,ROCKLAND,NY,10960,41.087078,-73.931658,75000,8.625295e+04,YES,8,0.0,0,EXCAVATION DAMAGE,EXCAVATION DAMAGE BY THIRD PARTY,NYACK COLLEGE IS A CAMPUS COMPRISED OF MULTIPL...,0.1,1969-01-01
7108,2/28/22,NaN,YES,YES,20210002,34674,1/4/21,ORIGINAL,31580,"MAGELLAN TERMINALS HOLDINGS, LP","ONE WILLIAMS CENTER, MD OTC-9 P.O. BOX 22186, ...",TULSA,OK,74172,12/5/20 9:56,NaN,NaN,2020,NaN,NaN,NaN,NaN,NaN,27.805754,-97.438549,430865,4.394575e+05,YES,7,0.0,0,OTHER INCIDENT